# optimization program

In [1]:
from pulp import *
import time
from datetime import timedelta

# iteration variables

     t    month

In [2]:
month = [0,1,2,3,4,5,6,7,8,9,10,11,12]
m = len(month)
demand = [0,21306,20477,18203,11106,5692,8616,9828,10273,14217,9520,18007,21662]
D = dict(zip(month,demand))

# Decision variables

   
    Pt   Workers assigned to production in month t
    Tt   Workers assigned to training in month t
    Lt   Laid off workers in month t
    Ft   Workers fired at the beginning of month t
    Rt   Total recruits hired at the beginning of month t
    It   Cumulative inventory at the end of month t
    St   Cumulative shortages (backlogs) at the end of month t
   


In [3]:
P = LpVariable.dicts('P', month, cat = 'Integer', lowBound = 0)
T = LpVariable.dicts('T', month, cat = 'Integer', lowBound = 0)
L = LpVariable.dicts('L', month, cat = 'Integer', lowBound = 0)
F = LpVariable.dicts('F', month, cat = 'Integer', lowBound = 0)
R = LpVariable.dicts('R', month, cat = 'Integer', lowBound = 0)
I = LpVariable.dicts('I', month, cat = 'Integer', lowBound = 0)
S = LpVariable.dicts('S', month, cat = 'Integer', lowBound = 0) 
W = LpVariable.dicts('W', month, cat = 'Integer', lowBound = 0)
X = LpVariable.dicts('X', month, cat = 'Integer', lowBound = 0)

# Parameters
     Wt   Total workers at the begining of the month t, before firing
     Xt   Number of units of C produced during month t

 # Objective function
 
$ \displaystyle MIN = 15000\sum_{t=1}^{12}P_t  + 15000\sum_{t=1}^{12} L_t+ 15000\sum_{t=1}^{12} F_t + 5000\sum_{t=1}^{12} R_t + 15000\sum_{t=1}^{12} T_t + 50\sum_{t=1}^{12} I_t + 50\sum_{t=1}^{12} S_t $ 

In [4]:
model = LpProblem("Aggregate Planning", LpMinimize)
model += 15000*sum(P[t] for t in month[1:13]) + 15000*sum(L[t] for t in month[1:13]) + 15000*sum(F[t] for t in month[1:13]) + 5000*sum(R[t] for t in month[1:13])  + 15000*sum(T[t] for t in month[1:13])+ 50*sum(I[t] for t in month[1:13]) + 50*sum(S[t] for t in month[1:13])

# Data

    W_1 = 10
    I_0 = 1000
    W_12 = 10
    I_12 = 1000
    S_12 = 0

In [5]:
model += I[0] == 1000
model += S[0] == 0
model += W[1] == 10
model += W[12] == 10
model += I[12] == 1000
model += S[12] == 0
#dummy
model += W[0] == 0
model += L[0] == 0
model += R[0] == 0
model += T[0] == 0
model += X[0] == 0
model += P[0] == 0
model += F[0] == 0

# Constraints

### Size of workforce  
$ \displaystyle W_t = W_{t-1} + R_{t-1} - F_{t-1} $

### Assignment of workforce
$ \displaystyle W_t = P_t + T_t + L_t + F_t $

### Training
$ \displaystyle R_t \leq 5T_t $


### Demand / Inventory balance
$ \displaystyle X_t + I_{t-1} = D_t + S_{t-1} + I_t - S_t $


### Production capacity
$ \displaystyle X_t \leq 1500P_t $


### Non-negativity constraints
$ \displaystyle P_t, T_t, L_t, F_t, R_t, I_t, S_t, X_t >= 0  \quad  ∀ t = 1, 2, 3, ..., 12$


### Integer constraints
$ \displaystyle P_t, T_t, L_t, F_t, R_t, I_t, S_t, X_t \quad are \, all \,  integers $


In [6]:
# Constraints
# Constraint 1
for t in month[2:13]:
    model += W[t] == W[t-1] + R[t-1] - F[t-1]

In [7]:
# Constraint 2
for t in month[1:13]:
    model += W[t] == P[t] + L[t] + F[t] + T[t]

In [8]:
# Constraint 3
for t in month[1:13]:
    model += R[t] <= 5*T[t]

In [9]:
#Constraint 4
for t in month[1:13]:
    model += X[t] + I[t-1] == D[t] + S[t-1] + I[t] - S[t]

In [10]:
# Constraint 5
for t in month[1:13]:
    model += X[t] <= 1500*P[t]

In [11]:
# save model to LP File
model.writeLP('AggregatePlanning.lp')
# view model
print(model)

Aggregate Planning:
MINIMIZE
15000*F_1 + 15000*F_10 + 15000*F_11 + 15000*F_12 + 15000*F_2 + 15000*F_3 + 15000*F_4 + 15000*F_5 + 15000*F_6 + 15000*F_7 + 15000*F_8 + 15000*F_9 + 50*I_1 + 50*I_10 + 50*I_11 + 50*I_12 + 50*I_2 + 50*I_3 + 50*I_4 + 50*I_5 + 50*I_6 + 50*I_7 + 50*I_8 + 50*I_9 + 15000*L_1 + 15000*L_10 + 15000*L_11 + 15000*L_12 + 15000*L_2 + 15000*L_3 + 15000*L_4 + 15000*L_5 + 15000*L_6 + 15000*L_7 + 15000*L_8 + 15000*L_9 + 15000*P_1 + 15000*P_10 + 15000*P_11 + 15000*P_12 + 15000*P_2 + 15000*P_3 + 15000*P_4 + 15000*P_5 + 15000*P_6 + 15000*P_7 + 15000*P_8 + 15000*P_9 + 5000*R_1 + 5000*R_10 + 5000*R_11 + 5000*R_12 + 5000*R_2 + 5000*R_3 + 5000*R_4 + 5000*R_5 + 5000*R_6 + 5000*R_7 + 5000*R_8 + 5000*R_9 + 50*S_1 + 50*S_10 + 50*S_11 + 50*S_12 + 50*S_2 + 50*S_3 + 50*S_4 + 50*S_5 + 50*S_6 + 50*S_7 + 50*S_8 + 50*S_9 + 15000*T_1 + 15000*T_10 + 15000*T_11 + 15000*T_12 + 15000*T_2 + 15000*T_3 + 15000*T_4 + 15000*T_5 + 15000*T_6 + 15000*T_7 + 15000*T_8 + 15000*T_9 + 0
SUBJECT TO
_C1: I_0 = 10

In [12]:
# solve the model
start_time = time.monotonic()
model.solve()
end_time = time.monotonic()
print("Status:",LpStatus[model.status])
print("Objective: ",value(model.objective))
print("Duration:",timedelta(seconds=end_time - start_time))

Status: Optimal
Objective:  3590450.0
Duration: 0:00:00.405000


### display optimized variable value

In [13]:
# create empty table
import pandas as pd
df = pd.DataFrame(index = range(0,13), columns=['month_ref','W','P','T','L','F','I','R','S','X','D'])
df = df.fillna(0)
# rearrange demand
demand = [0,21306,9520,18007,21662,20477,18203,11106,5692,8616,9828,10273,14217]
df['D'] = demand
df['month_ref'] = model.variables()[0:13]

In [14]:
# function to get values
import re
def val(col, var):
    x = pd.Series([0,0,0,0,0,0,0,0,0,0,0,0,0])
    i = 0
    for v in var:
        if(re.compile(col).match(v.name,0)):
            x[i] = v.varValue
            i = i+1
    return(x.values)

In [15]:
# store value in table
df['F'] = val('F', model.variables())
df['I'] = val('I', model.variables())
df['L'] = val('L', model.variables())
df['P'] = val('P', model.variables())
df['R'] = val('R', model.variables())
df['S'] = val('S', model.variables())
df['T'] = val('T', model.variables())
df['W'] = val('W', model.variables())
df['X'] = val('X', model.variables())

In [16]:
# view table
df

,month_ref,W,P,T,L,F,I,R,S,X,D
0,F_0,0,0,0,0,0,1000,0,0,0,0
1,F_1,10,8,2,0,0,0,9,8306,12000,21306
2,F_10,13,13,0,0,0,10669,0,0,19500,9520
3,F_11,13,10,0,0,3,7662,0,0,15000,18007
4,F_12,10,10,0,0,0,1000,0,0,15000,21662
5,F_2,19,19,0,0,0,0,0,283,28500,20477
6,F_3,19,13,0,0,6,0,0,0,18486,18203
7,F_4,13,8,0,0,5,0,0,0,11106,11106
8,F_5,8,4,3,0,1,0,0,0,5692,5692
9,F_6,7,6,1,0,0,0,1,0,8616,8616


In [17]:
# solve the model
start_time = time.monotonic()
model.solve(GLPK_CMD())
end_time = time.monotonic()
print("Status:",LpStatus[model.status])
print("Objective: ",value(model.objective))
print("Duration:",timedelta(seconds=end_time - start_time))

Status: Optimal
Objective:  3590450
Duration: 0:00:00.187000


In [18]:
# solve the model
start_time = time.monotonic()
model.solve(GUROBI())
end_time = time.monotonic()
print("Status:",LpStatus[model.status])
print("Objective: ",value(model.objective))
print("Duration:",timedelta(seconds=end_time - start_time))

Optimize a model with 72 rows, 117 columns and 225 nonzeros
Variable types: 0 continuous, 117 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [5e+01, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+04]
Presolve removed 37 rows and 55 columns
Presolve time: 0.00s
Presolved: 35 rows, 62 columns, 141 nonzeros
Variable types: 0 continuous, 62 integer (0 binary)

Root relaxation: objective 3.467809e+06, 31 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3467809.12    0   18          - 3467809.12      -     -    0s
H    0     0                    4596850.0000 3467809.12  24.6%     -    0s
H    0     0                    3636850.0000 3467809.12  4.65%     -    0s
     0     0 3535330.27    0   19 3636850.00 3535330.27  2.79%     -    0s
H    0     0                    3621300.0000 35

In [19]:
# solve the model
start_time = time.monotonic()
model.solve(CPLEX_CMD())
end_time = time.monotonic()
print("Status:",LpStatus[model.status])
print("Objective: ",value(model.objective))
print("Duration:",timedelta(seconds=end_time - start_time))

Status: Optimal
Objective:  3590449.9999999995
Duration: 0:00:00.203000


In [20]:
# solve the model
start_time = time.monotonic()
model.solve(CPLEX_PY())
end_time = time.monotonic()
print("Status:",LpStatus[model.status])
print("Objective: ",value(model.objective))
print("Duration:",timedelta(seconds=end_time - start_time))

Tried aggregator 2 times.
MIP Presolve eliminated 17 rows and 31 columns.
MIP Presolve added 11 rows and 11 columns.
Aggregator did 19 substitutions.
Reduced MIP has 47 rows, 78 columns, and 178 nonzeros.
Reduced MIP has 0 binaries, 78 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 9 rows and 9 columns.
MIP Presolve added 9 rows and 9 columns.
Reduced MIP has 47 rows, 78 columns, and 178 nonzeros.
Reduced MIP has 0 binaries, 78 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.07 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.13 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0  3467809.1152    25                3467809.1152       21         
  